# Python for Data Analysis - Workbook 5 (Aggregating and Grouping)

### Preliminaries

In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 
                   'key2': ['one', 'two', 'one', 'two', 'one'], 
                   'data1': np.random.randn(5), 
                  'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,1.161676,-0.382438
1,a,two,-1.666170,1.474923
2,b,one,-1.726285,0.521704
3,b,two,-0.459961,-0.998841
4,a,one,0.794591,0.142549


In [8]:
# Suppose we wanted to calculate the mean from data1 using keys from key1

grouped = df['data1'].groupby(df['key1'])
grouped # Creates special GroupBy object

In [9]:
# Can now calculate mean from this

grouped.mean()

key1
a    0.096699
b   -1.093123
Name: data1, dtype: float64

In [14]:
# Passing in multiple Series creates a GroupBy object with a hierarchical index

means = df['data1'].groupby([df['key1'], df['key2']]).mean()
print(means)
means.unstack()

key1  key2
a     one     0.978133
      two    -1.666170
b     one    -1.726285
      two    -0.459961
Name: data1, dtype: float64


key2,one,two
key1,,
a,0.978133,-1.666170
b,-1.726285,-0.459961


In [16]:
# Can also use any array that is the right length, not just Series in the same dataframe

states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -1.666170
            2006   -1.726285
Ohio        2005    0.350858
            2006    0.794591
Name: data1, dtype: float64

In [17]:
# Can also just pass column names

df.groupby('key1').mean()

# Note that key2 is dropped because it can't be aggregated (it is categorical)
# by default, all numeric columns are aggregated 

,data1,data2
key1,,
a,0.096699,0.411678
b,-1.093123,-0.238569


In [20]:
# Get number of items that end up being aggregated, though missing values from a group key will be excluded

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

#### Iterating over Groups

In [21]:
# GroupBy object supports iteration. It generates a sequence of 2-tuples containing the group name along with the 
# chunk of data

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.161676 -0.382438
1    a  two -1.666170  1.474923
4    a  one  0.794591  0.142549
b
  key1 key2     data1     data2
2    b  one -1.726285  0.521704
3    b  two -0.459961 -0.998841


In [22]:
# In the case of multiple keys, the first element in the tuple will be the tuple of key values

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.161676 -0.382438
4    a  one  0.794591  0.142549
a two
  key1 key2    data1     data2
1    a  two -1.66617  1.474923
b one
  key1 key2     data1     data2
2    b  one -1.726285  0.521704
b two
  key1 key2     data1     data2
3    b  two -0.459961 -0.998841


In [35]:
# One application for this is to create a dict of the data pieces as a one-liner

pieces = dict(list(df.groupby('key1')))
# pieces
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.726285,0.521704
3,b,two,-0.459961,-0.998841


In [40]:
# Default grouping happens on axis = 0, but can also do column-wise as well

grouped = df.groupby(df.dtypes, axis = 1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.161676 -0.382438
1 -1.666170  1.474923
2 -1.726285  0.521704
3 -0.459961 -0.998841
4  0.794591  0.142549
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [41]:
# Selecting only a subset of columns for groupby

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.119944
     two   1.474923
b    one   0.521704
     two  -0.998841

In [47]:
# Grouping with dicts

people = pd.DataFrame(np.random.randn(5, 5), columns = list('abcde'), index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan # add a few na values

print(people)

mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f':'orange'}

people.groupby(mapping, axis = 1).sum()

               a         b         c         d         e
Joe     1.073571  1.567554  1.219167 -0.209018 -0.471810
Steve   0.985780  0.083055 -0.362425 -0.723889  0.492432
Wes    -0.696793       NaN       NaN -0.788155 -1.535000
Jim    -0.520464 -0.111967  0.444795 -2.111839 -0.864139
Travis -0.408202 -0.381875  0.981163 -1.517212 -0.028128


,blue,red
Joe,1.010149,2.169315
Steve,-1.086313,1.561266
Wes,-0.788155,-2.231793
Jim,-1.667044,-1.496571
Travis,-0.536049,-0.818206


In [48]:
# Also works with Series

map_series = pd.Series(mapping)
print(map_series)
people.groupby(mapping, axis = 1).sum()


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object


,blue,red
Joe,1.010149,2.169315
Steve,-1.086313,1.561266
Wes,-0.788155,-2.231793
Jim,-1.667044,-1.496571
Travis,-0.536049,-0.818206


#### Grouping with Functions

In [49]:
# Grouping with functions is a more generic way of defining a group mapping vs a dict or series
# Any function passed as a group key will be called once per index value, w/ return values used as group names

people.groupby(len).sum()

,a,b,c,d,e
3,-0.143686,1.455587,1.663962,-3.109013,-2.870950
5,0.985780,0.083055,-0.362425,-0.723889,0.492432
6,-0.408202,-0.381875,0.981163,-1.517212,-0.028128


In [50]:
# Can also mix functions with arrays, dicts etc

key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.696793  1.567554  1.219167 -0.788155 -1.535000
  two -0.520464 -0.111967  0.444795 -2.111839 -0.864139
5 one  0.985780  0.083055 -0.362425 -0.723889  0.492432
6 two -0.408202 -0.381875  0.981163 -1.517212 -0.028128

#### Grouping by Index Level

In [57]:
# Can aggregate using only one level of an axis index

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names = ['city', 'tnr'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)
print(hier_df)
hier_df.groupby(level = 'city', axis = 1).count()

city        US                            JP          
tnr          1         3         5         1         3
0     1.411802 -1.412733  1.022808  0.832527 -0.314085
1     0.938582 -0.180290 -1.967836 -0.298716 -0.225837
2    -0.186946  2.138659  0.404939  0.701811  1.136043
3     0.007405 -1.147697  0.483667 -0.728572  0.517681


city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### Data Aggregation

In [70]:
# Can use predefined and optimized methods like count, sum, mean, std, min, max
# Or can call any method from the grouped object. 

df.groupby('key1')['data1'].quantile(0.9)

key1
a    1.088259
b   -0.586593
Name: data1, dtype: float64

In [71]:
# Can also pass in custom aggregation functions, but these will be slower than the pre-baked ones provided by the API

def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby('key1')['data1'].agg(peak_to_peak)

key1
a    2.827846
b    1.266324
Name: data1, dtype: float64

In [74]:
# A couple of examples
 
tips = pd.read_csv('pydata-book/examples/tips.csv')

# Add tip % to the total bill

tips['tip_pct'] = tips['tip'] / (tips['total_bill'])

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [78]:
grouped = tips.groupby(['day', 'smoker'])
grouped['tip_pct'].agg('mean') # can also pass in the name of the function as a string for descriptive statistics

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [79]:
# Passing a list of functions returns a dataframe with a list of columns w/ names taken from each of the functions

grouped['tip_pct'].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [81]:
# Can change the column names by passing in a list of 2-tuples as a mapping

grouped['tip_pct'].agg([('foo','mean'), ('bar','std'), ('baz', peak_to_peak)])

foo       bar       baz
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [83]:
functions = ['count', 'mean', 'max']

grouped[['tip_pct', 'total_bill']].agg(functions)

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [84]:
# Can also apply different functions to each column individually

grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [85]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [91]:
# Can also suppress the automated indexing behavior and melt the indexes back into the resulting df as columns

tips.groupby(['day', 'smoker'], as_index = False).mean() # can also just call reset_index on the result

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### The Apply Method - general split/apply/combine

In [92]:
# Example, select the top five tip_pct values by group

def top(df, n=5, column = 'tip_pct'):
    return df.sort_values(by = column)[-n:]

top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [93]:
# What if we group by smoker?

tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [98]:
# Can pass arguments to apply functions as well

tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [101]:
# The describe() method also returns a groupby object...and can be manipulated in a similar fashion

result = tips.groupby('smoker')['tip_pct'].describe()
print(result)
result.unstack('smoker')

        count      mean       std       min       25%       50%       75%  \
smoker                                                                      
No      151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   
Yes      93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   

             max  
smoker            
No      0.291990  
Yes     0.710345  


       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [104]:
# Groupby automatically uses the group keys as an index in the hierarchy, but we can suppress that behavior

tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


#### Quantile and Bucket Analysis

In [106]:
# Groupby + cut and qcut makes for easy bucket or quantile analysis

frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)

In [110]:
quartiles[:10]

0    (-1.483, 0.138]
1    (-1.483, 0.138]
2    (-1.483, 0.138]
3     (0.138, 1.758]
4    (-1.483, 0.138]
5     (0.138, 1.758]
6    (-1.483, 0.138]
7    (-1.483, 0.138]
8     (0.138, 1.758]
9    (-1.483, 0.138]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.11, -1.483] < (-1.483, 0.138] < (0.138, 1.758] < (1.758, 3.378]]

In [115]:
# Categorical object returned by cut can be passed directly into groupby
# Here's how we would combine to compute the summary statistics for the data2 column

def get_stats(group):
    return({'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()})

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.11, -1.483]",-2.055139,2.287701,62.0,0.167308
"(-1.483, 0.138]",-2.231525,2.887238,489.0,0.051885
"(0.138, 1.758]",-2.943179,2.791713,409.0,-0.003603
"(1.758, 3.378]",-1.667625,1.619606,40.0,0.085913


##### Groupby Example 1: Filling missing values with group-specific values

In [131]:
# Sometimes you want to handle na's differently for different columns of data. Groupby can help with that

states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan # Set some values as missing for example
data

Ohio          0.128497
New York     -0.555623
Vermont            NaN
Florida      -0.528744
Oregon        1.005982
Nevada             NaN
California    0.493904
Idaho              NaN
dtype: float64

In [123]:
data.fillna(data.mean())

Ohio          1.165969
New York     -0.817639
Vermont      -0.465578
Florida      -2.016265
Oregon       -0.416362
Nevada       -0.465578
California   -0.243595
Idaho        -0.465578
dtype: float64

In [135]:
data.groupby(group_key).mean()

East   -0.318623
West    0.749943
dtype: float64

In [136]:
# Fill NA values using group means

fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.128497
New York     -0.555623
Vermont      -0.318623
Florida      -0.528744
Oregon        1.005982
Nevada        0.749943
California    0.493904
Idaho         0.749943
dtype: float64

In [137]:
# Or use predefined fill values that vary by group

fill_values = {'East': 0.5, 'West': -1.}
fill_func = lambda g: g.fillna(fill_values[g.name])

In [138]:
data.groupby(group_key).apply(fill_func)

Ohio          0.128497
New York     -0.555623
Vermont       0.500000
Florida      -0.528744
Oregon        1.005982
Nevada       -1.000000
California    0.493904
Idaho        -1.000000
dtype: float64